In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed
from numpy.random import randint
import time

In [2]:
def leArquivo(caminho):
    #inicializando as variaveis
    rows = 0
    cols = 0
    #lendo os dados
    f = open(caminho,"r")

    #lendo a primeira linha
    count = 0
    A = []
    RHS = []
    utilidade = []
    upreenchida = False
    
    for linha in f:
        
        if linha =="\n":
            count = count + 1
            continue

        if count ==0:
            rows = int(linha)

        if count == 1:
            cols = int(linha)
            
        
        if count == 2:
            if upreenchida == False:
                for val in linha.split():
                    utilidade.append(float(val))
                upreenchida = True     
                    
            else:
                vetor =[]
                for val in linha.split():
                    vetor.append(float(val))
                A.append(vetor)

        if count == 3:
            for val in linha.split():
                RHS.append(float(val))

    f.close()
    return np.array(A),np.array(RHS),rows,cols,np.array(utilidade)

In [3]:
def doisOPTMelhor(a,x,b,u):    
    
    OMelhorObj =  RetornarValorDaFuncaoObj(x,u)
    OMelhorX = x.copy()
    xBarra = []
    firstimprove = False
    for i in range (col):
        j = i+1    
        while j < col:
            
            xBarra = x.copy().tolist()
            
            before = xBarra[:i]
            middle= xBarra[(i):(j)]
            middle.reverse()
            after=xBarra[(j):]
            
            xBarra = before + middle + after
            xBarra = list(map(int, xBarra))
            xBarra = np.array(xBarra)
            
            Obj = OMelhorObj
            h=i
            while h < (j):
                if OMelhorX[h] ==  xBarra[h]:
                    h=h+1  
                    continue
                    
                else:
                   
                    if xBarra[h]==1:
                        Obj = Obj + u[h]
                    else:
                        Obj = Obj - u[h]
                h=h+1                
            #print("''ObjDoisOptMelhor",OMelhorObj,"''ObjDoisOpt",Obj)
            if Obj > OMelhorObj:
                if EViavel(a,xBarra,b):
                    OMelhorObj = Obj
                    OMelhorX = xBarra.copy()
            j = j+1            
            
    return OMelhorObj,OMelhorX

In [4]:
def EViavel(a, x, b):
    eviavel= True 
    for i in range(row):
        total=0
        for j in range(col):
            total= total + (a[i,j]*x[j])
        if total> b[i]:
            eviavel= False 
            break 
    return eviavel

In [5]:
def RetornarValorDaFuncaoObj(x, u):
    total=0
    for j in range(col):
        total= total + (x[j]*u[j])
    return total 

In [6]:
def BuscaTabu(tempo,T,x, A, RHS, cols, rows, c):
    ini = time.time()
    achouT = time.time()
    
    melhorFO = RetornarValorDaFuncaoObj(c,x)
    melhorx= x.copy()
    vetorITabu = []
    vetorJTabu = []
    vetorJMaisTabu = []

    while True:
        jLinha=-1
        iLinha=-1
        JMaislinha = -1
        melhorFOtabu=-1
        melhorxtabu=[]

        for i in range(cols):
            j = i+1

            while j < cols-1:
                xx = melhorx.copy()
                indiceTabu= -1
                usouCriterioAspiracao= False
            
                for k in range(len(vetorITabu)):
                    if vetorITabu[k]==i and vetorJTabu[k]==j and  vetorJMaisTabu[k]==j+1:
                        indiceTabu=k
            
                if indiceTabu==-1:
                    xx[i],xx[j],xx[j+1]= 1-xx[i], 1-xx[j],1-xx[j+1]
                    #FOLinha= RetornarValorDaFuncaoObj(c, xx)
                    FOLinha = melhorFO
                    if melhorx[i] == xx[i] and melhorx[j] == xx[j] and melhorx[j+1] == xx[j+1]:
                        continue
                    else:
                        if xx[i]==1 :
                            FOLinha = FOLinha + c[i]
                        else:
                            FOLinha = FOLinha - c[i]      
                        if xx[j]==1 :
                            FOLinha = FOLinha + c[j]
                        else:
                            FOLinha = FOLinha - c[j]      
                        if xx[j+1]==1 :
                            FOLinha = FOLinha + c[j+1]
                        else:
                            FOLinha = FOLinha - c[j+1]      
                    
                    if FOLinha > melhorFOtabu: 
                        if EViavel(A,xx,RHS):
                            melhorxtabu= xx.copy()
                            melhorFOtabu= FOLinha
                            jLinha=j
                            iLinha=i
                            JMaislinha = j+1

                else:

                    xx[i],xx[j],xx[j+1]= 1-xx[i], 1-xx[j],1-xx[j+1]
                    #FOLinha= RetornarValorDaFuncaoObj(c, xx)
                    FOLinha = melhorFO
                    if melhorx[i] == xx[i] and melhorx[j] == xx[j] and melhorx[j+1] == xx[j+1]:
                        continue
                    else:
                        if xx[i]==1 :
                            FOLinha = FOLinha + c[i]
                        else:
                            FOLinha = FOLinha - c[i]      
                        if xx[j]==1 :
                            FOLinha = FOLinha + c[j]
                        else:
                            FOLinha = FOLinha - c[j]      
                        if xx[j+1]==1 :
                            FOLinha = FOLinha + c[j+1]
                        else:
                            FOLinha = FOLinha - c[j+1]      
                    if FOLinha > melhorFO: 
                        if EViavel(A,xx,RHS):
                            melhorxtabu= xx.copy()
                            melhorFOtabu= FOLinha
                            jLinha=j
                            iLinha=i
                            JMaislinha = j+1
                            usouCriterioAspiracao= True
                j=j+1                            

        if iLinha != -1:
            if(usouCriterioAspiracao ==0): 
                vetorITabu.append(iLinha) 
                vetorJTabu.append(jLinha) 
                vetorJMaisTabu.append(JMaislinha) 
                achouT = time.time()

                if len(vetorITabu) > T:
                    vetorITabu=vetorITabu[(1):]
                    vetorJTabu=vetorJTabu[(1):]
                    vetorJMaisTabu=vetorJMaisTabu[(1):]
        else:
            vetorITabu.append(iLinha) 
            vetorJTabu.append(jLinha) 
            vetorJMaisTabu.append(JMaislinha) 

        if melhorFOtabu> melhorFO:
            melhorFO = melhorFOtabu
            melhorx = melhorxtabu
            achouT = time.time()

        fim = time.time()
        if fim <= (ini+tempo):
            continue
        else: 
            break
        #(achouT-ini)    
    return melhorFO,melhorx

In [7]:
def HeuristicaGulosa(a,b,c,x):
    cc = c.copy()
    xx = x.copy() 
    Viavel = True
    maxIndex  = 0
    while Viavel:
        maxIndex = np.argmax(cc)
        xx[maxIndex] = 1
        cc[maxIndex] = 0 
        Viavel = EViavel(a,xx,b)
    xx[maxIndex] = 0
    return xx

In [8]:
def calculaVNS(tempo,T,A,RHS,row,col,Utility,x):
    ObjAnterior  = 0
    XAnterior = x.copy()
    XAnterior = HeuristicaGulosa(A, RHS, Utility, XAnterior)
    ObjAnterior=RetornarValorDaFuncaoObj(x,Utility)
    MelhorObj = ObjAnterior
    MelhorX = XAnterior
    tempototal = 0
    
    naoHouvemelhoria = True
    while(naoHouvemelhoria):
        ObjAnterior,XAnterior  = BuscaTabu(tempo,T,XAnterior, A, RHS, col, row, Utility)
        if ObjAnterior > MelhorObj:
            MelhorObj = ObjAnterior
            MelhorX = XAnterior.copy()
            print("TabuObj",MelhorObj)
                    
        else:
            ObjAnterior,XAnterior  = doisOPTMelhor(A,XAnterior,RHS, Utility)
            print("'DoisOptMelhor",MelhorObj,"DoisOptObjAnterior",ObjAnterior)
            if ObjAnterior > MelhorObj:
                MelhorObj = ObjAnterior
                MelhorX = XAnterior.copy()
                print("*DoisOptMelhor",MelhorObj)
            else:
                naoHouvemelhoria = False        
            
            
    print("*VNS - *Melhor Obj Final",MelhorObj,"*Melhor X Final",MelhorX)  

In [9]:
A = []
row = 0
col = 0
RHS = []
Utility  = []
x =[]

C5I10 = ""
C5I10 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C5I10.dat"
A,RHS,row,col,Utility = leArquivo(C5I10)
x = np.zeros(col) 
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)

C30I20 = ""
C30I20 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C30I20.dat"
A,RHS,row,col,Utility = leArquivo(C30I20)
x = np.zeros(col) 
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)

C150I200 = ""
C150I200 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C150I200.dat"
A,RHS,row,col,Utility = leArquivo(C150I200)
x = np.zeros(col) 
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)


C500I500 =""
C500I500 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C500I500.dat"
A,RHS,row,col,Utility = leArquivo(C500I500)
x = np.zeros(col) 
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)

C500I700 = ""
C500I700 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C500I700.dat"
A,RHS,row,col,Utility = leArquivo(C500I700)
x = np.zeros(col)
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)

C700I500 = ""
C700I500 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C700I500.dat"
A,RHS,row,col,Utility = leArquivo(C700I500)
x = np.zeros(col)
tempo= 10
T= int(col*0.3)
calculaVNS(tempo,T,A,RHS,row,col,Utility,x)


TabuObj 3.84807
'DoisOptMelhor 3.84807 DoisOptObjAnterior 3.84807
*VNS - *Melhor Obj Final 3.84807 *Melhor X Final [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
TabuObj 9.257930000000002
'DoisOptMelhor 9.257930000000002 DoisOptObjAnterior 9.257930000000002
*VNS - *Melhor Obj Final 9.257930000000002 *Melhor X Final [0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
TabuObj 89.80277999999997
TabuObj 90.28161999999998
TabuObj 90.57758
'DoisOptMelhor 90.57758 DoisOptObjAnterior 90.57757999999998
*VNS - *Melhor Obj Final 90.57758 *Melhor X Final [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 0. 1. 1.

In [10]:
xvalue = [[]]*0
vetor= [0,1,0,1,0,1,0,1,0,1]
vetordois= [0,1,2,3,4,5,6,7,8,9]
xvalue.append(vetor) 
xvalue.append(vetordois)
xvalue.append([])
xvalue

for i in range(3):
    if len(xvalue[i]) <= 0:
        print("nulo",xvalue[i])
    else:
        print("preenchido",xvalue[i])
        
print(xvalue)

preenchido [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
preenchido [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
nulo []
[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], []]


In [11]:
del xvalue[0]
print(xvalue)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], []]


In [12]:
[0,1,2,3,4,5,6,7,8,9]
[0,1,0,1,0,1,0,1,0,1]
xvalue.append(vetorxvalue)
vetorxvalue2 = [0,1,0,1,0,1,0,1,0,1]
xvalue.append(vetorxvalue2)
print(vetorxvalue)

NameError: name 'vetorxvalue' is not defined

In [ ]:
x = np.zeros(10)

for i in range(10):
    j = i+1
    while j < 9:
        xx = x.copy()
        xx[i],xx[j],xx[j+1]= 1-xx[i], 1-xx[j],1-xx[j+1]
        print(xx)
        j=j+1

In [ ]:
x = np.zeros(10)
j=0
while j < 10:
    xx = x.copy()
    xx[j],xx[j+1]= 1-xx[j],1-xx[j+1]
    
    print(xx)
    j=j+1

In [ ]:
teste = 733*0.3
teste = int(teste)
teste